# Segmenting and Clustering Neighborhoods in Toronto

1. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
2. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
3. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.
4. If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
5. Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
6. In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [16]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [10]:
wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = requests.get(wiki_url).text

In [90]:
soup = BeautifulSoup(html)

table = soup.find_all('table', class_="wikitable sortable")
df = pd.read_html(str(table))[0]

# clean empty rows by Borough
df = df[df['Borough']!='Not assigned']

# check duplicated rows by Postal Code
if df[df.duplicated(['Postal Code'], keep=False)].size > 0:
    raise Exception('This dataframe contains duplicated rows by Postal Code')
    
if df[df["Neighbourhood"]=='Not assigned'].size > 0:
    raise Exception('This dataframe contains Not assigned values in Neighbourhood column')

In [91]:
df.shape

(103, 3)